In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import tdt
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import xarray as xr

import kdephys.hypno as kh
import kdephys.pd as kpd
import kdephys.xr as kx
import kdephys.utils as ku
import kdephys.ssfm as ss

import ecephys.units as eu

import acr.subjects as subs
import ecephys

/home/kdriessen/github_t2/kdephys/kdephys/xr/utils.py:4: DeprecationWarning: Please use `gaussian_filter1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter1d


In [ ]:
import ecephys.units.io as uio

In [ ]:
min_fr, max_fr = (0, float("inf"))
min_depth, max_depth = (0, float("inf"))

ks_dir = Path("path/to/ks/dir")

ksdir = Path('/nvme/sorting/tdt/ACR_14-short-iso-NNXo/ks2_5_nblocks=1_8s-batches/')
sorting = uio.load_sorting_extractor(ksdir, good_only=False, drop_noise=True, selection_intervals={'fr': (min_fr, max_fr), 'depth': (min_depth, max_depth)})

In [ ]:
trn = sorting.get_all_spike_trains()

In [ ]:
sorting.get_unit_ids()

In [ ]:
cluster_info = uio.get_cluster_info(ksdir)
cluster_info = cluster_info[
    cluster_info.cluster_id.isin(sorting.get_unit_ids())
]

In [ ]:
all_spike_times = ecephys.units.get_spike_times_list(sorting, sorting.get_unit_ids())

In [ ]:
sdf = ecephys.units.spikeinterface_sorting_to_dataframe(sorting)

In [ ]:
s2 = sdf.loc[np.logical_and(sdf['t'] > 150, sdf['t'] < 300)]
s2

In [ ]:
quick = sdf.where(sdf.cluster_id == 3).dropna()

In [ ]:
q2 = quick.iloc[0:2000]

In [ ]:
bp = '/Volumes/opto_loc/Data/ACR_14/ACR_14-short1-iso/'
chan = [2, 5, 7, 12]
lfp = kx.io.get_data(bp, 'LFPo', t1=0, t2=1000, channel=chan)

In [ ]:
from plotly.subplots import make_subplots
import plotly.graph_objects as go
# duplicate above matplotlib plot in plotly
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)
fig.add_trace(go.Scatter(x=lfp.sel(channel=12).time, y=lfp.sel(channel=12).values, mode='lines', name='LFP'), row=1, col=1)
fig.add_trace(go.Scatter(x=s2.t, y=s2.cluster_id, mode='markers', marker=dict(size=5)), row=2, col=1)
fig

# Unit + Field Potential Inspector

In [1]:
%matplotlib inline
%reload_ext autoreload
%autoreload 2

import numpy as np
import pandas as pd
import tdt
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
import yaml
import xarray as xr

import kdephys.hypno as kh
import kdephys.pd as kpd
import kdephys.xr as kx
import kdephys.utils as ku
import kdephys.ssfm as ss

import acr.subjects as subs
import acr
import plotly.express as px
import ecephys.units as eu

/home/kdriessen/github_t2/kdephys/kdephys/xr/utils.py:4: DeprecationWarning: Please use `gaussian_filter1d` from the `scipy.ndimage` namespace, the `scipy.ndimage.filters` namespace is deprecated.
  from scipy.ndimage.filters import gaussian_filter1d


In [2]:
rec = 'short2-iso'
root = '/nvme/sorting/tdt/' 
ss582 = 'ACR_14-test-short-582/ks2_5_nblocks=1_8s-batches'
ss5104 = 'ACR_14-test-short-5104/ks2_5_nblocks=1_8s-batches'

In [3]:
dp = '/Volumes/opto_loc/Data/ACR_14/short2-iso-LFP_.nc'

In [4]:
fp = xr.load_dataarray(dp)

In [ ]:
d1 = Path(f'/nvme/sorting/tdt/test-short-582/ks2_5_nblocks=1_8s-batches/')
d2 = Path(f'/nvme/sorting/tdt/test-short-5104/ks2_5_nblocks=1_8s-batches/')
ex1 = eu.load_sorting_extractor(d1)
ex2 = eu.load_sorting_extractor(d2)

In [ ]:
ci1 = eu.get_cluster_info(d1)
ci1

In [ ]:
df1 = eu.spikeinterface_sorting_to_dataframe(ex1)
df2 = eu.spikeinterface_sorting_to_dataframe(ex2)

In [ ]:
dti = pd.to_timedelta(df1.t.values, unit='s') + fp.datetime.values.min()
df1['datetime'] = dti

In [ ]:
s2 = sdf.loc[np.logical_and(sdf['t'] > 150, sdf['t'] < 300)]

In [ ]:
def unit_fp_inspector(fp, spk, channel=14, t1=0, t2=120):
    fp = fp.swap_dims({'datetime': 'time'}).sel(time=slice(t1, t2), channel=channel)
    spk = spk.loc[np.logical_and(spk['t'] > t1, spk['t'] < t2)]
    return fp, spk

In [ ]:
f, s = unit_fp_inspector(fp, df1, channel=14, t1=0, t2=120)

In [ ]:
fp.channel.values

In [ ]:

# plotly subplots
from plotly.subplots import make_subplots
import plotly.graph_objects as go
fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.02)
fig.add_trace(go.Scatter(x=f.time, y=f.values, name='LFP'), row=1, col=1)
fig.add_trace(go.Scatter(x=s.t, y=s.cluster_id, mode='markers', marker=dict(size=2)), row=2, col=1)

In [6]:
def spectro_data(data, chan, f_range=slice(0, 35)):
    spg = kx.spectral.get_spextrogram(data)
    ax = kx.plots.spectro_plotter(spg, chan=chan, f_range=f_range)
    return ax
    